**Xception Net**

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/kaggle/input/xceptionnet/xceptionmodel.h5')

**Single Image**

In [ ]:
test_image = image.load_img('/kaggle/input/real-dataset-for-deepfake-test/deepfake real dataset/female/Copy of DBCL-DBIT-IT-118.JPGface3.jpg', target_size=(256, 256, 3))
plt.imshow(test_image)
test_image_arr = image.img_to_array(test_image)
test_image_arr = np.expand_dims(test_image, axis=0)
test_image_arr = test_image_arr/255.


result = loaded_model.predict(test_image_arr)

plt.title(f"This image is {100 * (1 - result[0][0]):.2f}% Fake and {100 * result[0][0]:.2f}% Real.")

**Multiple Images**

In [ ]:
import os
import numpy as np
import csv
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load the saved model
loaded_model = load_model('/kaggle/input/xceptionnet/xceptionmodel.h5')

# Directory containing the images to be tested
image_dir = '/kaggle/input/real-dataset-for-deepfake-test/deepfake real dataset/male'

# Output CSV file to store the results
output_csv = 'image_predictions.csv'

# Create a CSV file to store the results
with open(output_csv, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Image_Path', 'Prediction', 'Is_Fake'])

    # Set the threshold to classify an image as 'fake'
    threshold = 0.6

    for root, _, files in os.walk(image_dir):
        for filename in files:
            if filename.endswith('.jpg'):
                # Load and preprocess the image
                image_path = os.path.join(root, filename)
                test_image = image.load_img(image_path, target_size=(256, 256, 3))
                test_image_arr = image.img_to_array(test_image)
                test_image_arr = np.expand_dims(test_image_arr, axis=0)
                test_image_arr = test_image_arr / 255.

                # Make predictions using the loaded model
                result = loaded_model.predict(test_image_arr)

                # Determine if the image is classified as 'fake' based on the threshold
                is_fake = result[0][0] >= threshold

                # Write the results to the CSV file
                csv_writer.writerow([image_path, result[0][0], is_fake])

                # Optionally, plot the image with the prediction
                plt.imshow(test_image)
                plt.title(f"This image is {100 * (1 - result[0][0]):.2f}% Fake and {100 * result[0][0]:.2f}% Real.")
                plt.show()

print(f"Predictions saved to {output_csv}")


In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

# Output CSV file with the results
output_csv = 'image_predictions.csv'

# Load CSV data and classify as 'Real' or 'Fake'
with open(output_csv, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip the header row
    is_fake_list = []  # To store whether an image is classified as 'fake'

    for row in csv_reader:
        image_path, _, is_fake = row
        is_fake_list.append(is_fake == 'True')  # Convert 'True' and 'False' strings to boolean

# Count the number of 'Real' and 'Fake' classifications
num_real = sum(1 for is_fake in is_fake_list if not is_fake)
num_fake = sum(1 for is_fake in is_fake_list if is_fake)

# Plot a bar chart with counts on top of the bars
categories = ['Real', 'Fake']
counts = [num_real, num_fake]

plt.bar(categories, counts, color=['blue', 'red'])
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Real Male Images Classified as Real vs. Fake')

# Add counts on top of the bars
for i, count in enumerate(counts):
    plt.text(i, count, str(count), ha='center', va='bottom', fontsize=12)

plt.show()
